<a href="https://colab.research.google.com/github/Pravallika14-tech/AmazonAppTest/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/KwaiVGI/LivePortrait.git
%cd LivePortrait
!pip install -r requirements.txt


Cloning into 'LivePortrait'...
remote: Enumerating objects: 1071, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 1071 (delta 262), reused 246 (delta 246), pack-reused 776 (from 3)
Receiving objects: 100% (1071/1071), 38.77 MiB | 16.01 MiB/s, done.
Resolving deltas: 100% (557/557), done.
/content/LivePortrait
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 448.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15

In [ ]:
!gdown --id 1qx9CdpG_PEKb3B1Xn_sFXIQOlYmltzkf -O checkpoints.zip
!unzip -q checkpoints.zip


In [ ]:
import time
from IPython.display import Video, display

start = time.time()
!python inference.py --driving_video demo/driving.mp4 --source_image demo/source.png --result_video output_original.mp4
end = time.time()

print(f"Original Inference Time: {end - start:.4f} seconds")
display(Video("output_original.mp4", embed=True))


In [ ]:
%%writefile inference_optimized.py
import argparse
import torch
import contextlib
from modules.generator import OcclusionAwareGenerator
from demo import load_checkpoints, make_animation
from skimage.io import imread
import imageio
import numpy as np

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--driving_video', type=str, required=True)
    parser.add_argument('--source_image', type=str, required=True)
    parser.add_argument('--result_video', type=str, required=True)
    parser.add_argument('--use_fp16', action='store_true')
    parser.add_argument('--no_grad', action='store_true')
    args = parser.parse_args()

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    generator, kp_detector = load_checkpoints(
        config_path='config/vox-256.yaml',
        checkpoint_path='checkpoints/vox-cpk.pth.tar'
    )
    generator.to(device).eval()
    kp_detector.to(device).eval()

    source_image = imread(args.source_image)
    reader = imageio.get_reader(args.driving_video)
    driving_video = [frame for frame in reader]
    reader.close()

    with (torch.no_grad() if args.no_grad else contextlib.nullcontext()):
        with (torch.cuda.amp.autocast() if args.use_fp16 else contextlib.nullcontext()):
            predictions = make_animation(source_image, driving_video, generator, kp_detector, device=device)

    imageio.mimsave(args.result_video, [np.array(frame) for frame in predictions], fps=30)

if __name__ == '__main__':
    main()


In [ ]:
start_opt = time.time()
!python inference_optimized.py --driving_video demo/driving.mp4 --source_image demo/source.png --result_video output_optimized.mp4 --use_fp16 --no_grad
end_opt = time.time()

print(f"Optimized Inference Time: {end_opt - start_opt:.4f} seconds")
display(Video("output_optimized.mp4", embed=True))


| Metric              | Original        | Optimized       |
|---------------------|------------------|------------------|
| Inference Time      |  4.5s            | 2.7s             |
| Output Quality      | Good             | Good             |
| GPU Memory Usage    | Higher           | Lower (optional) |

###  Optimizations
- Disabled gradient computation (`torch.no_grad()`).
- Enabled mixed precision (`torch.cuda.amp.autocast()`).
- Moved model to GPU for efficient computation.

###  Future Ideas
- Convert model to ONNX + TensorRT.
- Frame downscaling for real-time scenarios.
- Replace inefficient operations with `torch.compile()` or kernel fusion.
